# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

ModuleNotFoundError: No module named 'unityagents'

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [ ]:
env = UnityEnvironment(file_name='Reacher_1.app')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
max_steps = 10
step_count = 0
while step_count <= max_steps:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    step_count += 1
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
env_info = env.reset(train_mode=True)[brain_name]

In [ ]:
actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
actions = np.clip(actions, -1, 1)

In [ ]:
env_info = env.step(actions)[brain_name]           # send all actions to tne environment
next_states = env_info.vector_observations

Critic

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from collections import OrderedDict, deque, namedtuple
import random
import sys
import copy

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
REPLAY_MEMORY_SIZE = int(1e6)
REPLAY_BATCH_SIZE = 128
LR_ACTOR = 5e-3
LR_CRITIC = 1e-3
TAU = 1e-3
GAMMA = 0.99
WEIGHT_DECAY = 0
NOISE_DECAY = 0.9999
UPDATE_EVERY = 64

In [ ]:
def fanin_init(size, fanin=None):
    fanin = fanin or size[0]
    v = 1. / np.sqrt(fanin)
    return torch.Tensor(size).uniform_(-v, v)

def get_randome_seed():
    np.random.seed()
    seed = np.random.randint(0, sys.maxsize)
    return seed

In [ ]:
class ReplayBuffer:
    
    def __init__(self, action_size, buffer_size, batch_size, seed):
        
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", 
                                     field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, states, actions, rewards, next_states, dones):
        for (state, action, reward, next_state, done) in zip(states, actions, rewards, next_states, dones): 
            e = self.experience(state, action, reward, next_state, done)
            self.memory.append(e)
        
    def sample(self):
        experiences = random.sample(self.memory, k=self.batch_size)
        
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        
        return (states, actions, rewards, next_states, dones)
    
    def __len__(self):
        return len(self.memory)

In [ ]:
class ActorNet(nn.Module):
    
    def __init__(self, state_size, action_size, hidden_layers=[256], seed=0):
        super(ActorNet, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, hidden_layers[0])
        self.fc2 = nn.Linear(hidden_layers[0], action_size)
        self.init_weights()
        self.to(device)

    def forward(self, states):
        x = states
        x = torch.relu(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        return x
    
    def init_weights(self):
        self.fc1.weight.data = fanin_init(self.fc1.weight.data.size())
        self.fc2.weight.data = self.fc2.weight.data.uniform_(-3e-3, 3e-3)

In [ ]:
class CriticNet(nn.Module):
    
    def __init__(self, state_size, action_size, hidden_layers=[256, 128, 64], seed=0):
        super(CriticNet, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, hidden_layers[0])
        self.fc2 = nn.Linear(hidden_layers[0] + action_size, hidden_layers[1])
        self.fc3 = nn.Linear(hidden_layers[1], hidden_layers[2])
        self.fc4 = nn.Linear(hidden_layers[2], 1)
        self.init_weights()
        self.to(device)
    
    def forward(self, states, actions):
        x = F.leaky_relu(self.fc1(states))
        y = actions
        x = torch.cat([x, y], -1)
        x = F.leaky_relu(self.fc2(x))
        x = F.leaky_relu(self.fc3(x))
        x = self.fc4(x)
        return x
    
    def init_weights(self):
        self.fc1.weight.data = fanin_init(self.fc1.weight.data.size())
        self.fc2.weight.data = fanin_init(self.fc2.weight.data.size())
        self.fc3.weight.data = fanin_init(self.fc3.weight.data.size())
        self.fc4.weight.data = self.fc4.weight.data.uniform_(-3e-3, 3e-3)

In [ ]:
class ActorCriticNets:
    
    def __init__(self, state_size, action_size, seed=0):
        super(ActorCriticNets, self).__init__()
        self.seed = seed
        self.actor = ActorNet(state_size=state_size, action_size=action_size, seed=seed)
        self.critic = CriticNet(state_size=state_size, action_size=action_size, seed=seed)

In [ ]:
class DDPGAgent():
    
    def __init__(self, state_size, action_size, train_mode=True, seed=None):
        super(DDPGAgent, self).__init__()
        self.seed = random.seed(seed)
        self.state_size = state_size
        self.action_size = action_size
        self.train_mode = train_mode
        self.local_networks = ActorCriticNets(state_size, action_size, seed)
        self.target_networks = ActorCriticNets(state_size, action_size, seed)
        self.hard_update()
                
        self.actor_opt = torch.optim.Adam(self.local_networks.actor.parameters(), lr=LR_ACTOR)
        self.actor_opt_scheduler = torch.optim.lr_scheduler.StepLR(self.actor_opt, step_size=500, gamma=0.9)
        
        self.critic_opt = torch.optim.Adam(self.local_networks.critic.parameters(), lr=LR_CRITIC, weight_decay=WEIGHT_DECAY)
        self.critic_opt_scheduler = torch.optim.lr_scheduler.StepLR(self.critic_opt, step_size=500, gamma=0.9)
        
        self.memory = ReplayBuffer(action_size=action_size, 
                                   buffer_size=REPLAY_MEMORY_SIZE, 
                                   batch_size=REPLAY_BATCH_SIZE, 
                                   seed=seed)
        self.states = None
        self.scores = np.zeros(num_agents)
        self.update_every = UPDATE_EVERY
        self.steps = 0
        
        self.noise = OUNoise(seed=seed, size=action_size)
        self.normalizer = Normalizer()

    
    def reset(self, env, brain_name=brain_name):
        self.scores = np.zeros(num_agents)
        self.steps = 0
        env_info = env.reset(train_mode=self.train_mode)[brain_name] 
        self.states = env_info.vector_observations
        
    def select_action(self):
        states = torch.from_numpy(self.states).float().to(device)
        self.local_networks.actor.eval()
        with torch.no_grad():
            actions = self.local_networks.actor(states).cpu().data.numpy()
        self.local_networks.actor.train()
        
        if self.train_mode:
            actions += self.noise.sample()
        actions = np.clip(actions, -1, 1)
        return actions
        
    def act(self, env, brain_name=brain_name):                
        actions = self.select_action()
        env_info = env.step(actions)[brain_name]  
        next_states = env_info.vector_observations
        rewards = env_info.rewards
        dones = env_info.local_done
        return self.states, actions, rewards, next_states, dones
        
    def step(self, env, gamma, tau):        
        states, actions, rewards, next_states, dones = self.act(env)
        self.scores += rewards
        self.memory.add(states, actions, rewards, next_states, dones)
        self.states = next_states
        self.steps = (self.steps + 1) % self.update_every

        if (len(self.memory) > self.memory.batch_size) and (self.update_every == 0):
            experiences = self.memory.sample()
            # Update local nets
            self.learn(experiences, gamma)
            # soft update
            self.soft_update(tau)
        
        return dones
        
    def learn(self, experiences, gamma):
        states, actions, rewards, next_states, dones = experiences
        
        # calculate critic loss
        target_next_actions = self.target_networks.actor(next_states)
        target_next_critic_value = self.target_networks.critic(next_states, target_next_actions)
        target_critic_value = rewards + gamma * (1 - dones) * target_next_critic_value
        local_critic_value = self.local_networks.critic(states, actions)
        critic_loss = F.mse_loss(local_critic_value, target_next_critic_value)   
        
        # update critic
        self.critic_opt.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.local_networks.critic.parameters(), 1)
        self.critic_opt.step()
        
        # calculate policy loss
        actions_pred = self.local_networks.actor(states)
        policy_loss = -self.local_networks.critic(states, actions_pred).mean()
        
        # it's your turn, actor
        self.actor_opt.zero_grad()
        policy_loss.backward()
        self.actor_opt.step()
        
    def soft_update(self, tau):            
        for target_param, local_param in zip(self.target_networks.actor.parameters(), 
                                             self.local_networks.actor.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)
        
        for target_param, local_param in zip(self.target_networks.critic.parameters(), 
                                             self.local_networks.critic.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)
            
    def hard_update(self):            
        for target_param, local_param in zip(self.target_networks.actor.parameters(), 
                                             self.local_networks.actor.parameters()):
            target_param.data.copy_(local_param.data)
        
        for target_param, local_param in zip(self.target_networks.critic.parameters(), 
                                             self.local_networks.critic.parameters()):
            target_param.data.copy_(local_param.data)
    
    def save(self, filename):
        torch.save(self.local_networks.actor.state_dict(), filename + '_actor.pth')
        torch.save(self.local_networks.critic.state_dict(), filename + '_critic.pth')

    def load(self, filename):
        actor_state_dict = torch.load(filename + '_actor', map_location=lambda storage, loc: storage)
        self.local_networks.actor.load_state_dict(actor_state_dict)
        
        critic_state_dict = torch.load(filename + '_critic', map_location=lambda storage, loc: storage)
        self.local_networks.critic.load_state_dict(critic_state_dict)

In [ ]:
class Normalizer:
    
    def __init__(self):
        pass
    
    def normalize_batch(self, batch):
        std = batch.mean() + 1e-4
        mean = batch.std()
        return (batch - mean) / std

In [ ]:
# import multiprocessing as mp
# import sys

# class ProcessWrapper(mp.Process):
#     STEP = 0
#     RESET = 1
#     EXIT = 2
#     SPECS = 3
#     def __init__(self, pipe, task):
#         mp.Process.__init__(self)
#         self.pipe = pipe
#         self.task = lambda: DDPGAgent(state_size, action_size)
    
#     def run(self):
#         task = self.task()
#         while True:
#             op, data = self.pipe.recv()
#             if op == self.STEP:
#                 self.pipe.send(task.step(data))
#             elif op == self.RESET:
#                 self.pipe.send(task.reset())
#             elif op == self.EXIT:
#                 self.pipe.close()
#                 return
#             elif op == self.SPECS:
#                 self.pipe.send([task.state_dim, task.action_dim, task.name])
#             else:
#                 raise Exception('Unknown command')
        

# class ProcessTask:
#     def __init__(self):
#         self.pipe, worker_pipe = mp.Pipe()
#         self.worker = ProcessWrapper(worker_pipe, task)
#         self.worker.start()
#         self.pipe.send([ProcessWrapper.SPECS, None])
#         self.state_size, self.action_size = self.pipe.recv()

# class ParallelAgents(mp.Process):
#     def __init__(self, num_agents):
#         self.num_agents = num_agents
#         self.tasks = [ProcessTask() for _ in range(num_agents)]
    
#     def step(self):
#         results = [task.step(action) for task, action in zip(self.tasks, actions)]
#         results = map(lambda x: np.stack(x), zip(*results))
#         return results

In [ ]:
class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.2):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(len(x))])
        self.state = x + dx
        return self.state
    
    def std_decay(self):
        self.sigma = self.sigma * NOISE_DECAY

TODO:
1. Normalize rewards, states
2. Clip gradients
3. Action Noise

### Train

In [ ]:
# agents = [DDPGAgent(state_size, action_size) for i in range(num_agents)]
agent = DDPGAgent(state_size, action_size, train_mode=True, seed=10)

In [ ]:
scores = []
scores_window = deque(maxlen=100)
MAX_STEPS = 1100
episodes = 200

In [ ]:
for i in range(1, episodes + 1):
    
    agent.reset(env)
    num_steps = 0
    done = False
    while (not done) and (num_steps <= MAX_STEPS):
        dones = agent.step(env, GAMMA, TAU)
        done = dones[0]
        num_steps += 1

    scores.append(agent.scores.mean())
    scores_window.append(agent.scores.mean())
    agent.noise.std_decay()
    
    if np.mean(scores_window) > 30:
        print('Episode {}: Average score of consecutive 100 episodes: {:.2f}'.format(i, np.mean(scores_window)))
        agent.save('checkpoint')
    elif i % 10 == 0:
        print('Episode {} score: {:.2f}...'.format(i, agent.scores.mean()))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(np.arange(len(scores)), scores)
# plt.hlines(30, 0, 100, colors='green', linestyle='dashed')
# plt.hlines(np.mean(scores), 0, 100, colors='red', linestyle='dashed')
plt.ylabel('Score')
plt.xlabel('Episodes')

### Evaluate